# Prepare to launch TensorFlow Serving

In [2]:
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import pathlib
import glob
import os
import sys

sys.path.append("../")

In [12]:
from tfserving_manager.model_server_config import TFServingModelServerConfig

TensorFlow Serving requires a special model config file in the protobuf message format. This file can be generated using the below functions. In the example below, the base path for the model is specified according to what it would be inside the Docker container where we will run TensorFlow Serving. Note, that the model base path does not include the model version sub-directory, as the model version can be specified inside the model config file. If no version is specified, the latest model is used.

In [13]:
models = glob.glob(os.path.join("..", "models", "build", "*"))

In [14]:
config_file = TFServingModelServerConfig()
config_file.initialise_blank_config()

for model in models:
    fp = pathlib.Path(model)
    if fp.is_dir():
        model_name = fp.stem
        base_path = os.path.join("/", "models", model_name, "")
        config_file.add_model(model_name=model_name, base_path=base_path)

In [11]:
config_file.save_protobuf(os.path.join("..", "models", "build", "model_config.proto"))

Launch TensorFlow Serving in a Docker container using the following command from the root directory:

```
docker run \
-p 8500:8500 \
-p 8501:8501 \
--mount type=bind,source="$(pwd)/models/build/",target=/models \
tensorflow/serving \
--model_config_file=/models/model_config.proto
```